In [0]:
# mount
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# get git repo
%cd /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages
!rm -rf /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCVLab
!git clone https://github.com/pmcgrath249/DeepCVLab.git

# copy data such that can remove whole repo without loosing the data
!cp -r /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/data /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCVLab/deepcvlab/data

# permanently change dir 
%cd /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCVLab/deepcvlab

# install waymo dataset utils in utils; https://github.com/waymo-research/waymo-open-dataset/blob/master/tutorial/tutorial.ipynb
!cd utils && rm -rf waymo-od > /dev/null
!cd utils && git clone https://github.com/waymo-research/waymo-open-dataset.git waymo-od
!cd utils/waymo-od && git branch -a
!cd utils/waymo-od && git checkout remotes/origin/r1.0
!pip3 install --upgrade pip
!pip3 install waymo-open-dataset

# install requirements
!cd /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCVLab/ && pip3 install -r requirements.txt

# install own package
!cd /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCVLab/ && python3 -m pip install .


# directly retrieve data from google
# https://medium.com/@philipplies/transferring-data-from-google-drive-to-google-cloud-storage-using-google-colab-96e088a8c041
'''
from google.colab import auth
auth.authenticate_user()
project_id = 'nifty-depth-246308'
!gcloud config set project {project_id}
!gsutil ls
bucket_name = 'medium_demo_bucket_190710'
!gsutil -m cp -r /content/mnt/My\ Drive/Colab\ Notebooks/DeepCV_Packages/* gs://{bucket_name}/
'''

In [ ]:
'''
for waymo to work make sure tf is version 1.x
for tensorboard to work enable 3rd party cookies in your browser
'''
# import
%cd /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCVLab/
from deepcvlab.utils.Dense_U_Net_lidar_helper import get_config
from deepcvlab.agents.Dense_U_Net_lidar_Agent import Dense_U_Net_lidar_Agent as Dense_U_Agent

config = get_config()
config.agent.max_epoch = 100

# use tensorboard to visualize
%load_ext tensorboard
%tensorboard --logdir {os.path.join(*config.dir.summary.split('/')[-2:])}

# start training
agent = Dense_U_Agent(config=config, torchvision_init=True)
agent.run()
agent.finalize()